# Keras Examples

### Why Should I Care?

Keras is a high level API for Tensorflow (and as a standalone) that 
simplifies many of the tasks involved in implementing a ML model. As a
student, incorporating Keras and related high level API entities will:

1. **Save you time** - features that would take time to implement may already
be included in Keras.
2. **Improve readability** - High level APIs can facilitate the writing of clean
and readable code, allowing you to focus on learning the core concepts.
3. **Compatibility** - Since high level APIs add a level of abstraction onto the
underlying operations, your code will be more resilient to changes made to these
underlying operations.

## Importing Images 

This was one of the first use cases that sold me on the value of Keras. For image
classification or regression, you will often need to import images with some
underlying directory structure. My initial approach to handling this on an assigment
was to write my own functions to traverse the directory and parse images / classes
as required for the assignment. I eventually discovered that Keras provides high
level APIs to automate this entire process using a few lines of code.

**Note**: See [documentation](https://keras.io/preprocessing/image/) for reference

A minimal version of Tiny Imagenet is included in this directory, 
where only one image of each class is included. 

We can view the directory structure...

In [ ]:
import os
DATASET_ROOT = './tiny_imagenet_minimal'
for root, dirs, files in os.walk(DATASET_ROOT):
    for name in files:
        print(os.path.join(root, name))

We see that the training set contains 200 directories corresponding to the 200 
classes in Tiny Imagenet. Within each directory is an `images` directory that
contains the individual `JPEG` images of that class.

We will begin by creating an `ImageDataGenerator` object as follows.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        data_format='channels_last',
        samplewise_center=True,
        samplewise_std_normalization=True,
        horizontal_flip=True,
        rescale=1./255
)

The passed arguments:

 * `samplewise_center=True` - Normalize to zero mean
 * `samplewise_std_normalization=True` - Normalize to unit variance
 * `horizontal_flip=True` - Flip images
 * `data_format=True` - Generator should yield images with channels on axis 0
 * `rescale=1./255` - Rescale 8 bit images to a float on [0, 1]

The constructor also allows one to specify a custom preprocessing function to run 
after the above operations. We have just defined a generator, but no processing
has actually been performed. Tensorflow has added the file reading operations and
the pre-processing steps we specified to the computational graph.

Next we use the `flow_from_directory` method which automatically
interprets the file structure of the training set and returns an iterator
over training files. Our preprocessing operations will be applied as files are pulled 
from this iterator as tuples of image label pairs. The arguments are mostly trivial. 
We specify a `class_mode='sparse'` such that an argmax integer is produced for labels, 
rather than a one hot vector. 

In [ ]:
train_generator = train_datagen.flow_from_directory(
        DATASET_ROOT,
        target_size=(64, 64),
        batch_size=4,
        class_mode='sparse'
)

We get a nice output stating the number of images and classes that were detected.

We can then plot the images.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def plot_images(img_iter, rows=2, cols=2):
    fig=plt.figure(figsize=(8, 8))
    img_iter = img_iter[:rows*cols]
    for i, img in enumerate(img_iter):
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(img)
    plt.show()
    return img_iter

img_bat, label_bat = train_generator.next()
_ = plot_images(img_bat)

Note that some of the images look strange - the yielded images have had their
preprocessing operations applied.

Later demos will show exactly how to apply this workflow to training models.
The brief explanation is that the generator we created can be used for any Keras 
call that accepts a generator as an input argument. Similarly, we can use this
generator to offer training examples as inputs for TFRecord generation.